In [ ]:
import numpy as np

class GaussianNB:
    def __init__(self, var_smoothing=1e-9):
        self.var_smoothing = var_smoothing

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_features = X.shape[1]
        n_classes = len(self.classes_)

        self.theta_ = np.zeros((n_classes, n_features))
        self.var_ = np.zeros((n_classes, n_features))
        self.class_prior_ = np.zeros(n_classes)

        for i, c in enumerate(self.classes_):
            X_c = X[y == c]
            self.theta_[i] = np.mean(X_c, axis=0)
            self.var_[i] = np.var(X_c, axis=0) + self.var_smoothing
            self.class_prior_[i] = X_c.shape[0] / X.shape[0]

    def predict(self, X):
        log_likelihood = []
        for i in range(len(self.classes_)):
            prior = np.log(self.class_prior_[i])
            likelihood = -0.5 * np.sum(np.log(2 * np.pi * self.var_[i]))
            likelihood -= 0.5 * np.sum(((X - self.theta_[i]) ** 2) / self.var_[i], axis=1)
            log_likelihood.append(prior + likelihood)

        return self.classes_[np.argmax(np.array(log_likelihood).T, axis=1)]